# Imports

In [ ]:
import importlib
import pandas as pd
import numpy as np

import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils

# Load data

In [ ]:
cone_optim_folder = os.path.join('..', '..', 'step1a_optimize_cones', 'optim_data')
cbc_optim_folder = os.path.join('..', '..', 'step2a_optimize_cbc', 'optim_data')

In [ ]:
os.listdir(cone_optim_folder)

In [ ]:
sorted(os.listdir(cbc_optim_folder))

In [ ]:
cell2folder = {
    'Cone':  os.path.join(cone_optim_folder, 'optimize_cone_submission2'),
    'OFF':  os.path.join(cbc_optim_folder, 'optimize_OFF_submission2'),
    'ON':   os.path.join(cbc_optim_folder, 'optimize_ON_submission2'),
}

In [ ]:
final_model_outputs = {}

for cell, folder in cell2folder.items():
    final_model_outputs[cell] = data_utils.load_var(os.path.join(folder, 'post_data', 'final_model_output.pkl'))

# Compute peak times

In [ ]:
def compute_peak_delays(cell, params_dict_list, rng=(-0.3,0.3), ignore_rec_times=[]):
    trace_list = [final_model_outputs[cell]['Target'], final_model_outputs[cell]['iGlu']]
    time_list =  final_model_outputs[cell]['Time-Target']
    color_list = ['r', 'g']
    label_list = ['target', 'fit']

    peak_data = plot_peaks.compare_peaks_in_traces(
        trace_list=trace_list,
        time_list=time_list,
        params_dict_list=params_dict_list,
        color_list=color_list,
        label_list=label_list,
        plot=True,
        xlims=xlims,
        mode=2,
        rng=rng,
        figsize=(12,4),
        ignore_rec_times=ignore_rec_times, # Ignore noisy parts.
    )
        
    return peak_data

In [ ]:
import plot_peaks
importlib.reload(plot_peaks)
xlims = [(0,32), (1, 4), (4,6), (11, 13), (13, 15), (15, 17), (17, 19), (20, 22), (22, 25), (25, 27), (27, 31)]

In [ ]:
cell_peak_data = {}

## Cone

In [ ]:
np.random.seed(13214)

cell ='Cone'

params_dict_list = [{'prom': 0.1}, {'prom': 0.1}]

cell_peak_data[cell] = compute_peak_delays(cell, params_dict_list, ignore_rec_times=[(16, 23)])

## OFF

In [ ]:
np.random.seed(13214)

cell ='OFF'

params_dict_list = [{'prom': 0.2}, {'prom': 0.1}]

cell_peak_data[cell] = compute_peak_delays(cell, params_dict_list, ignore_rec_times=[(16, 21)])

## ON

In [ ]:
np.random.seed(13214)

cell ='ON'

params_dict_list = [{'prom': 0.1}, {'prom': 0.03}] 

cell_peak_data[cell] = compute_peak_delays(cell, params_dict_list, ignore_rec_times=[(16, 21)])

## Summarize

In [ ]:
# Print all peaks with > 100 ms dt

summarized_peak_data = {}

for cell, peak_data in cell_peak_data.items():
    summarized_peak_data[cell] = {}
    print()
    print(cell)
    print("Num pos. peaks", len(cell_peak_data[cell][-1]['fit']['pos']))
    print("Num neg. peaks", len(cell_peak_data[cell][-1]['fit']['neg']))
    
    for pn in ['pos', 'neg', 'all']:
        summarized_peak_data[cell][pn] = {}
        
        for typ in ['dt', 't0', 't1', 'A0', 'A1']:
            summarized_peak_data[cell][pn][typ] = []
        
        for peak_dt, peak_t0, peak_t1, peak_A0, peak_A1 in cell_peak_data[cell][-1]['fit'][pn]:
            summarized_peak_data[cell][pn]['dt'].append(peak_dt)
            summarized_peak_data[cell][pn]['t0'].append(peak_t0)
            summarized_peak_data[cell][pn]['t1'].append(peak_t1)
            summarized_peak_data[cell][pn]['A0'].append(peak_A0)
            summarized_peak_data[cell][pn]['A1'].append(peak_A1)
            
            if (np.abs(peak_dt) > 0.1) and (pn == 'all'):
                print(peak_dt)

In [ ]:
data_utils.make_dir('preprocessed_data')
data_utils.save_var(summarized_peak_data, 'preprocessed_data/summarized_peak_data.pkl')

# Extract some data for text

## Peak times

In [ ]:
from datetime import datetime

In [ ]:
lines = []

lines.append('%' + str(datetime.now()) + '\n')

for cell in ['ON', 'OFF']:
        
    for peak_type in ['pos', 'neg', 'all']:
        lines.append("\\newcommand\\delay" + cell + peak_type + "Mean{" +"{:.1f}".format(\
                                             np.mean(summarized_peak_data[cell][peak_type]['dt'])*1e3) + "}\n")
        lines.append("\\newcommand\\delay" + cell + peak_type + "Median{" + "{:.1f}".format(\
                                             np.median(summarized_peak_data[cell][peak_type]['dt'])*1e3) + "}\n")
lines

In [ ]:
data_utils.make_dir('text_data')
with open('text_data/peak_delay_measure.tex', 'w') as f:
    f.writelines(lines)

## Number of compartments.

In [ ]:
n_comps_lines = []

n_comps_lines.append('%' + str(datetime.now()) + '\n')

for cell, folder in cell2folder.items():
    if 'cbc' in folder:
        n_cpl_dict = data_utils.load_var(os.path.join(folder, 'n_cpl_dict.pkl'))
        for mode, n_comps in n_cpl_dict.items():
            n_comps_lines.append("\\newcommand\\ncomps" + cell + mode + "{" + str(n_comps) + "}\n")

n_comps_lines

In [ ]:
with open('text_data/n_comps.tex', 'w') as f:
    f.writelines(n_comps_lines)